# 查看Pipeline支持的任务类型

In [ ]:
from transformers.pipelines import SUPPORTED_TASKS

In [ ]:
from pprint import pprint
pprint(SUPPORTED_TASKS.keys())

In [ ]:
for k, v in SUPPORTED_TASKS.items():
    print(k, v)

# Pipeline的创建与使用方式

In [ ]:
from transformers import pipeline, QuestionAnsweringPipeline

## 根据任务类型直接创建Pipeline, 默认都是英文的模型

In [ ]:
pipe = pipeline("text-classification")

In [ ]:
pipe(["very good!", "vary bad!"])

## 指定任务类型，再指定模型，创建基于指定模型的Pipeline

In [ ]:
# https://huggingface.co/models
pipe = pipeline("text-classification", model="uer/roberta-base-finetuned-dianping-chinese")

In [ ]:
pipe("我觉得不太行！")

## 预先加载模型，再创建Pipeline

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# 这种方式，必须同时指定model和tokenizer
model = AutoModelForSequenceClassification.from_pretrained("uer/roberta-base-finetuned-dianping-chinese")
tokenizer = AutoTokenizer.from_pretrained("uer/roberta-base-finetuned-dianping-chinese")
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)

In [ ]:
pipe("我觉得不太行！")

In [ ]:
pipe.model.device

In [ ]:
import torch
import time
times = []
for i in range(100):
    torch.cuda.synchronize()
    start = time.time()
    pipe("我觉得不太行！")
    torch.cuda.synchronize()
    end = time.time()
    times.append(end - start)
print(sum(times) / 100)

## 使用GPU进行推理

In [ ]:
pipe = pipeline("text-classification", model="uer/roberta-base-finetuned-dianping-chinese", device=0)

In [ ]:
pipe.model.device

In [ ]:
import torch
import time
times = []
for i in range(100):
    torch.cuda.synchronize()
    start = time.time()
    pipe("我觉得不太行！")
    torch.cuda.synchronize()
    end = time.time()
    times.append(end - start)
print(sum(times) / 100)

## 确定Pipeline参数

In [ ]:
qa_pipe = pipeline("question-answering", model="uer/roberta-base-chinese-extractive-qa")

In [ ]:
qa_pipe

In [ ]:
QuestionAnsweringPipeline

In [ ]:
qa_pipe(question="中国的首都是哪里？", context="中国的首都是北京", max_answer_len=1)

# 其他Pipeline示例

In [ ]:
checkpoint = "google/owlvit-base-patch32"
detector = pipeline(model=checkpoint, task="zero-shot-object-detection")

In [ ]:
import requests
from PIL import Image

url = "https://unsplash.com/photos/oj0zeY2Ltk4/download?ixid=MnwxMjA3fDB8MXxzZWFyY2h8MTR8fHBpY25pY3xlbnwwfHx8fDE2Nzc0OTE1NDk&force=true&w=640"
im = Image.open(requests.get(url, stream=True).raw)
im

In [ ]:
predictions = detector(
    im,
    candidate_labels=["hat", "sunglasses", "book"],
)
predictions

In [ ]:
from PIL import ImageDraw

draw = ImageDraw.Draw(im)

for prediction in predictions:
    box = prediction["box"]
    label = prediction["label"]
    score = prediction["score"]
    xmin, ymin, xmax, ymax = box.values()
    draw.rectangle((xmin, ymin, xmax, ymax), outline="red", width=1)
    draw.text((xmin, ymin), f"{label}: {round(score,2)}", fill="red")

im

# Pipeline背后的实现

In [ ]:
from transformers import *
import torch

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("uer/roberta-base-finetuned-dianping-chinese")
model = AutoModelForSequenceClassification.from_pretrained("uer/roberta-base-finetuned-dianping-chinese")

In [ ]:
input_text = "我觉得不太行！"
inputs = tokenizer(input_text, return_tensors="pt")
inputs

In [ ]:
res = model(**inputs)
res

In [ ]:
logits = res.logits
logits = torch.softmax(logits, dim=-1)
logits

In [ ]:
pred = torch.argmax(logits).item()
pred

In [ ]:
model.config.id2label

In [ ]:
result = model.config.id2label.get(pred)
result

In [1]:
import os 
os.environ["http_proxy"] = "http://127.0.0.1:7890"
os.environ["https_proxy"] = "http://127.0.0.1:7890"

In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-7B-Chat")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-7B-Chat")

d:\anaconda\envs\roboflow\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sjf19\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


ValueError: Tokenizer class QWenTokenizer does not exist or is not currently imported.